<a href="https://colab.research.google.com/github/CDU-data-science-team/zero-shot/blob/main/hf_T_training_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

##Import needed library

In [ ]:
import transformers
import torch
print(transformers.__version__) #print the tranformer version
from transformers import AutoModelForSequenceClassification, DistilBertForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer   #the API for training model in transormer since PyTorch does not provide a training loop
# from time import time

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

4.11.2


In [ ]:
# # Needed to mount google drive on Colab platform
# from google.colab import drive
# drive.mount('/content/drive')

# # To used code to  upload files on Colab platform
# # from google.colab import files
# # files.upload()

Mounted at /content/drive


##Loading and split the dataset

In [ ]:
url = 'https://raw.githubusercontent.com/CDU-data-science-team/pxtextmining/main/datasets/text_data.csv'
data = pd.read_csv(url, usecols=['feedback', 'label'],  encoding='utf-8')#, nrows=100)
print(f'{data.head()} \n\nno. of feedbacks: {len(data)} \n')
df = data.loc[:8999,:]         # the remaining data is USED FOR TESTING final prediction

#  take a sample to easily test pipeline
# df = df.sample(1000).reset_index(drop=True)

#fill missing value has this causes runtime error while fiting the model 
print(f'Missing values\n{df.isna().sum()}')
df['feedback'].fillna('Nothing', inplace = True)
print(df.isna().sum())
print(f'{df.head()} \n\nno. of feedbacks: {len(df)} \n')

# Needed Function and Classes

In [ ]:
# Turn the labels and encodings into a Dataset object (using pytorch). 
class feedbkDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

# This will allow us to feed batches of sequences into the model at the same time)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])      # encode the label before doing this
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
### Define a function for evaluating the model
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

#'weighted' produced higher metrics than 'macro'
def compute_metrics(pred):
    r"""The function that will be used to compute metrics at evaluation. Must take a
    `transformers.EvalPrediction` and return a dictionary string to metric values
    Arguments:
    pred (:obj:`transformers.EvalPrediction`):
        model prediction from `transformers.EvalPrediction`
    Returns:
    :obj:`dict`: 
        dictionary string to metric values
    """ 
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')  # set average to 'weighted' (to take label imbalance into account.) | 'macro' (to not take label imbalance into account)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

#Preprocessing the data

1. Encode the label

2. tokenize the text feature

3. Combine the label and text together and convert them into a Dataset object
  (`torch.utils.data.Dataset` or `torch.utils.data.IterableDataset` object)

In [ ]:
# # Needed to map label2id and id2label in the model configuration (Very useful while using model in Zeroshot or test classification pipeline)
label2id = {}
id2label = {}
for v, k in enumerate(sorted( df.label.unique())):
  label2id[k] = v
  id2label[v] = k

### Define the model

In [ ]:
# Define model_checkpoint 
model_checkpoint = "distilbert-base-uncased"
# model_checkpoint = 'valhalla/distilbart-mnli-12-9'

# Define the Tokenizer the test feauture 
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, num_labels=len(set(df.label)))

# Define the model
# model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
# model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=len(set(df.label)))
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, id2label=id2label, label2id = label2id, num_labels=len(set(df.label)))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

In [ ]:
# Encode the label and create label2id and id2label - to map label2id and id2label in the model configuration (possible to be able to use model for Zeroshot)
df['label'] = LabelEncoder().fit_transform(df.label)

# Split the dataset
seed = 0
train_texts, train_labels = list(df.feedback), list(df.label)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2, random_state=seed) # create validation set
# train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size=.2, random_state=seed)   # create validation set 

print(f'No of feedbacks in\nTrain set: {len(train_texts)}\nValidation set: {len(val_texts)}') #\nTest set: {len(test_texts)} 

# (truncation=True, padding=True will ensure that all of our sequences are padded to the same length and 
# are truncated to be no longer than model’s maximum input length (512 in this case). 
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
# test_encodings = tokenizer(test_texts, truncation=True, padding=True)

train_dataset = feedbkDataset(train_encodings, train_labels)
val_dataset = feedbkDataset(val_encodings, val_labels)
# test_dataset = feedbkDataset(test_encodings, test_labels)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


No of feedbacks in
Train set: 7200
Validation set: 1800


The steps above prepared the datasets in the way that the trainer expected. Now all we need to do is create a model to fine-tune, define the TrainingArguments and instantiate a Trainer.

#Fine-tuning the model

### Instantiate a TrainingArguments 
to hold all the hyperparameters we can tune for the Trainer


In [ ]:
# metric_name = 'accuracy'
training_args = TrainingArguments(
    output_dir='/output',             # output directory
    num_train_epochs=2,               # total number of training epochs (2 appeared to be the best from trial)
    per_device_train_batch_size=16,   # batch size per device during training
    per_device_eval_batch_size=64,    # batch size for evaluation
    learning_rate=3e-5,               # (3e-5 appeared to be the best from trial)
    warmup_steps=500,                 # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # strength of weight decay
    evaluation_strategy = 'epoch',
    eval_steps = 10,                  # useful if evaluation_strategy="steps"`
    do_eval = True,                   # to run eval on the dev set.
    # do_train = True,                  # to train the model.
    # label_names = list(label2id.keys()), # The list of keys in our dictionary of inputs that correspond to the labels.
    # logging_steps=10,
    # logging_dir='./logs',             # directory for storing logs
)

In [ ]:
print(f'No of training Samples: {len(train_dataset)}')
training_args.device          # return the device used by this process
# training_args.n_gpu         # the number of GPUs used by this process

### Instantiate a trainer

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics,     # used to compute metrics at evaluation
    tokenizer=tokenizer,                 # to preprocess the dataset and make it easier to rerun an uninterrupted training or reuse the fine-tuned model.
)

### Train the model 
**(took 6hrs to train on 3,200/1,000 comments of train/validation set on CPU(RAM: 12GB))**

In [ ]:
trainer.train()   # will use GPU as long as you have access to a GPU (i.e to run script with gpu, no addition code is needed)

***** Running training *****
  Num examples = 7200
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 900


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.860766,0.737222,0.713945,0.715620,0.737222
2,1.334400,0.741012,0.761667,0.750380,0.742986,0.761667


***** Running Evaluation *****
  Num examples = 1800
  Batch size = 64
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to /output/checkpoint-500
Configuration saved in /output/checkpoint-500/config.json
Model weights saved in /output/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /output/checkpoint-500/tokenizer_config.json
Special tokens file saved in /output/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1800
  Batch size = 64
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` par

TrainOutput(global_step=900, training_loss=1.0565401543511286, metrics={'train_runtime': 1288.9808, 'train_samples_per_second': 11.172, 'train_steps_per_second': 0.698, 'total_flos': 1907768667340800.0, 'train_loss': 1.0565401543511286, 'epoch': 2.0})

In [ ]:
# %%time
# trainer.train()   # will use GPU as long as you have access to a GPU (i.e to run script with gpu, no addition code is needed)

***** Running training *****
  Num examples = 5760
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1800


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.258200,1.278319,0.594444,0.515906,0.484146,0.594444


***** Running Evaluation *****
  Num examples = 1800
  Batch size = 64
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to /output/checkpoint-500
Configuration saved in /output/checkpoint-500/config.json
Model weights saved in /output/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /output/checkpoint-500/tokenizer_config.json
Special tokens file saved in /output/checkpoint-500/special_tokens_map.json


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.258200,1.278319,0.594444,0.515906,0.484146,0.594444
2,0.844500,0.882645,0.724444,0.699981,0.695374,0.724444
3,0.761400,0.792243,0.748333,0.734659,0.722686,0.748333
4,0.763800,0.786020,0.755556,0.741524,0.730436,0.755556
5,0.524100,0.799447,0.755556,0.743179,0.736241,0.755556


***** Running Evaluation *****
  Num examples = 1800
  Batch size = 64
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to /output/checkpoint-1000
Configuration saved in /output/checkpoint-1000/config.json
Model weights saved in /output/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /output/checkpoint-1000/tokenizer_config.json
Special tokens file saved in /output/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1800
  Batch size = 64
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division

CPU times: user 45min 24s, sys: 9.93 s, total: 45min 34s
Wall time: 45min 43s


TrainOutput(global_step=1800, training_loss=0.9262501242425707, metrics={'train_runtime': 2743.7985, 'train_samples_per_second': 10.496, 'train_steps_per_second': 0.656, 'total_flos': 3815537334681600.0, 'train_loss': 0.9262501242425707, 'epoch': 5.0})

### Run model evaluation (on the test data - if available)

No need for this if no test data because `.evaluate()` uses self.eval_dataset by default and if compute metrics has been assigned to `Trainer` object the evaluation metrics on eval_data set will be returned anyways with train()

In [ ]:
#no need for this if no test data because .evaluate() uses self.eval_dataset by default and if compute metrics has been assigned to train() the evaluation metrics on eval_data set will be returned anyways
# trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples = 1440
  Batch size = 64


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'epoch': 5.0,
 'eval_accuracy': 0.7493055555555556,
 'eval_f1': 0.7318775678531758,
 'eval_loss': 0.8104530572891235,
 'eval_precision': 0.7262888029624507,
 'eval_recall': 0.7493055555555556,
 'eval_runtime': 36.9351,
 'eval_samples_per_second': 38.987,
 'eval_steps_per_second': 0.623}

# Save the FineTuned Model
Save a model and its configuration file to a directory, so that it can be re-loaded using the
        ``:func:~transformers.PreTrainedModel.from_pretrained`` class method.

In [ ]:
save_path = '/content/drive/MyDrive/Colab Notebooks/best_model/'
trainer.save_model(save_path) # Will save the model (and tokenizer if specified when initializing the trainer object), so you can reload it using from_pretrained()

# Use below codes to save the model and tokenizer seperately
# model.save_pretrained(save_path)   # same as above but only save the model
# tokenizer.save_pretrained(save_path) # same as above but save only the tokenizer

Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/best_model/
Configuration saved in /content/drive/MyDrive/Colab Notebooks/best_model/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/best_model/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab Notebooks/best_model/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab Notebooks/best_model/special_tokens_map.json
